# DII Analysis Template

This notebook provides a template for calculating and analyzing Dietary Inflammatory Index (DII) scores.

**Instructions:**
1. Replace `YOUR_DATA.csv` with your actual data file
2. Run all cells
3. Customize the analysis as needed

---


## 1. Setup


In [ ]:
# Install the package if needed
# !pip install git+https://github.com/strathlab-data/DII.git

import pandas as pd
import numpy as np
from dii import (
    calculate_dii,
    calculate_dii_detailed,
    get_available_nutrients,
    load_nutrient_data,
    summarize_input_data,
)

print("DII Calculator loaded successfully!")


## 2. Load Your Data

Replace the path below with your actual data file.


In [ ]:
# ============================================
# MODIFY THIS PATH TO YOUR DATA FILE
# ============================================
DATA_PATH = "YOUR_DATA.csv"

# Load the data
df = pd.read_csv(DATA_PATH)
print(f"Loaded {len(df)} participants")
df.head()


## 3. Check Data Coverage


In [ ]:
# See which DII nutrients are in your data
summary = summarize_input_data(df)

print(f"Data Coverage: {summary['coverage']:.1f}%")
print(f"\nNutrients found ({len(summary['nutrients_found'])}):") 
for n in summary['nutrients_found']:
    print(f"  ✓ {n}")

if summary['nutrients_missing']:
    print(f"\nNutrients missing ({len(summary['nutrients_missing'])}):") 
    for n in summary['nutrients_missing'][:10]:  # Show first 10
        print(f"  ✗ {n}")
    if len(summary['nutrients_missing']) > 10:
        print(f"  ... and {len(summary['nutrients_missing']) - 10} more")


## 4. Calculate DII Scores


In [ ]:
# Calculate DII scores
results = calculate_dii(df)

# Add scores back to original data
df['DII_score'] = results['DII_score']

print("DII Score Summary:")
print(df['DII_score'].describe())


## 5. Visualize Distribution


In [ ]:
# Use the built-in visualization (requires: pip install matplotlib)
from dii import plot_dii_distribution

plot_dii_distribution(df)


## 6. Categorize Participants


In [ ]:
# Create DII categories
def categorize_dii(score):
    if pd.isna(score):
        return 'Missing'
    elif score < -1:
        return 'Anti-inflammatory'
    elif score <= 1:
        return 'Neutral'
    else:
        return 'Pro-inflammatory'

df['DII_category'] = df['DII_score'].apply(categorize_dii)

# Summary table
category_summary = df['DII_category'].value_counts()
category_pct = df['DII_category'].value_counts(normalize=True) * 100

print("DII Category Distribution:")
print("=" * 40)
for cat in ['Anti-inflammatory', 'Neutral', 'Pro-inflammatory']:
    if cat in category_summary.index:
        print(f"{cat:<20} {category_summary[cat]:>6} ({category_pct[cat]:>5.1f}%)")


## 7. Detailed Analysis (Optional)

Get per-nutrient contributions to understand what's driving DII scores.


In [ ]:
# Get detailed breakdown and visualize
detailed = calculate_dii_detailed(df)

# Use built-in nutrient contribution plot
from dii import plot_nutrient_contributions
plot_nutrient_contributions(detailed)


## 8. Export Results


In [ ]:
# ============================================
# MODIFY OUTPUT PATHS AS NEEDED
# ============================================

# Simple output (just DII scores)
df[['DII_score', 'DII_category']].to_csv('dii_scores.csv', index=False)
print("Saved: dii_scores.csv")

# Detailed output (all calculations)
detailed.to_csv('dii_detailed.csv', index=False)
print("Saved: dii_detailed.csv")


---

## Citation

If you use this tool in your research, please cite:

```
Clark, T., & Strath, L. (2025). DII Calculator: A Python Implementation 
of the Dietary Inflammatory Index. GitHub. https://github.com/strathlab-data/DII
```

And the original DII methodology:

```
Shivappa, N., Steck, S. E., Hurley, T. G., Hussey, J. R., & Hébert, J. R. (2014). 
Designing and developing a literature-derived, population-based dietary 
inflammatory index. Public Health Nutrition, 17(8), 1689-1696.
```
